# Change to working directory

In [ ]:
cd /usr/src/app/yolov7

In [ ]:
!#pwd should be /usr/src/app/yolov7
!pwd

# Import libraries

In [ ]:
import os, multiprocessing

# Training 

There are 6 different models:
1. combined_positive_aug_640
2. combined_positive_aug_448
3. india_positive_aug_704
4. jp_usa_positive_aug_640
5. japan_positive_aug_608
6. norway_1280

- aug = contains data augmentation
- positive = if mentioned implies only those images were considered which had one or more road damage instances. Images with no instances were not considered.
- 448, 608, 1280 => image size

Download the weights of these models from the following link and place them inside `runs/train`.
LINK: https://drive.google.com/drive/folders/12yA1zrtD7tdo8-0ww0A_UxOzFuIj2TRF?usp=sharing


## Training

In [ ]:
weight_list = ['yolov7x.pt']*2 + ['yolov7-w6_training.pt'] + ['yolov7x.pt']*2 + ['yolov7-w6_training.pt']
workers = multiprocessing.cpu_count()
epochs = 55
imgs_list = [640, 448, 704, 640, 608, 1280]
bs_list = [16, 32, 20, 16, 16, 8]

cfg_folders = ['combined_positive_aug', 'combined_positive_aug', 'india_positive_aug', 'jp_us_positive_aug', 'japan_positive_aug', 'norway']

for wt, cfg_folder, imgs, bs in zip(weight_list, cfg_folders, imgs_list, bs_list):
    train_py = 'train_aux.py' if wt == 'yolov7-w6_training.pt' else 'train.py'
    print('---------------------')
    data = f'runs/cfg/{cfg_folder}/{cfg_folder}.yaml'
    name = f'train_{os.path.basename(data)[:-5]}_{imgs}'
    s_train = f'python {train_py} --weights {wt} --data {data} --epochs {epochs} --img-size {imgs} {imgs} --batch-size {bs} --cache-images --image-weights --device 0 --workers {workers} --name {name}'
    print(s_train)
    os.system(s_train)

# Inference

- The following code runs 5 inference models which achieve the best F1 score reported for each leaderboard.
- The results of inference is saved as .txt file in the format asked by orgainser in the folder runs/submissions.
- 'combined_without_drone' contains all the test images for easy reference for combined leaderboard.

In [ ]:
country_list = ['combined_without_drone', 'India', 'Japan', 'United_States', 'Norway']

# train_combined_positive_aug_640
ca_best = 'runs/train/train_combined_positive_aug_640/weights/best.pt'
ca_e24 = 'runs/train/train_combined_positive_aug_640/weights/epoch_024.pt'

# train_combined_positive_aug_448
ca_448_best = 'runs/train/train_combined_positive_aug_448/weights/best.pt'

# train_india_positive_aug_704
in_704_best = 'runs/train/train_india_positive_aug_704/weights/best.pt'
in_704_e24 = 'runs/train/train_india_positive_aug_704/weights/epoch_024.pt'

# train_japan_positive_aug_608
jpaa_best = 'runs/train/train_japan_positive_aug_608/weights/best.pt'
jpaa_e24 = 'runs/train/train_japan_positive_aug_608/weights/epoch_024.pt'

# train_jp_us_positive_aug_640
jual_best = 'runs/train/train_jp_us_positive_aug_640/weights/best.pt'
jual_e24 = 'runs/train/train_jp_us_positive_aug_640/weights/epoch_024.pt'

weight_list = [f'{ca_best} {ca_e24} {ca_448_best}', f'{in_704_best} {in_704_e24} {ca_best}', f'{jpaa_best} {jpaa_e24} {ca_best} {ca_e24}', f'{jual_best} {jual_e24} {ca_best} {ca_e24}',
f'{ca_best} {ca_e24} {ca_448_best}' ]
name_list = ['Combined', 'India', 'Japan', 'USA', 'Norway']

e = [True]*5
imgs = [640]*5
iou = 1
conf_list = [0.45, 0.4, 0.45, 0.4, 0.33]
nosave_flag = True
for wt, name, ensemble, imgsize, conf, country in zip(weight_list, name_list, e, imgs, conf_list, country_list):
    print('-----------------------')
    clower = country.lower()
    if country == 'combined_without_drone':
        source = f'../../mydataset/{country}'
    else:
        source = f'../../mydataset/{country}/test/images'
    notrace = '--no-trace' if ensemble else ''
    nosave = '--nosave' if nosave_flag else ''

    # tta_ag => test time augmentation, agnostic-nms
    name_detect = f'{name}_tta_ag_c{str(int(conf*100))}_i{str(int(iou*100))}_{imgsize}'
    print(name_detect)

    s = f'python detect.py --weights {wt} --source {source} --conf-thres {conf} --iou-thres {iou}  --img-size {imgsize} --augment --agnostic-nms --device 0 {nosave} --save-txt --name {name_detect} {notrace}'
    print(s)
    os.system(s)

    labelsrc = f'runs/detect/{name_detect}/labels'
    s_convert = f'python runs/convert_yolo_to_subformat.py --image_source {source} --label_source {labelsrc}'
    print(s_convert)
    os.system(s_convert)